# User interface

In [3]:
from nilearn.plotting import plot_anat, show
import ipywidgets as widgets
import utils
import os
from importlib import reload
from nilearn import plotting
from nilearn import image as nli
reload(utils)
import nibabel as nib
import numpy as np
from ipywidgets import HBox, VBox
import numpy as np
from nilearn.plotting import plot_anat, show
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [5]:
# Data input

# This block will collect database info

widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False   
)

Text(value='Hello World', description='String:', placeholder='Type something')

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# Display initial data_dict
data_dict = {
    "Dataset": "fc",
    "Participants": "1,2,3,4",
    "Runs": "1,2,3,4",
    "Otro": ""
}

# Function to display markdown text
def display_markdown():
    markdown_text = f"""
    ### Experiment Data
    **Dataset:** {data_dict['Dataset']}  \n
    **Participants:** {data_dict['Participants']}  \n
    **Runs:** {data_dict['Runs']}  \n
    **Otro:** {data_dict['Otro']}  
    """
    display(Markdown(markdown_text))

# Initial display
display_markdown()

# Create widgets for input
input_widgets = {
    "Dataset": widgets.Text(value=data_dict['Dataset'], description='Dataset:'),
    "Participants": widgets.Text(value=data_dict['Participants'], description='Participants:'),
    "Runs": widgets.Text(value=data_dict['Runs'], description='Runs:'),
    "Otro": widgets.Text(value=data_dict['Otro'], description='Otro:')
}

update_button = widgets.Button(description="Update")
cancel_button = widgets.Button(description="Cancel")

# Display widgets
for widget in input_widgets.values():
    display(widget)
display(update_button)
display(cancel_button)

# Update function
def update_data(b):
    # declare data_dict as global variable
    global data_dict

    clear_output(wait=True)  # Clears the current output and waits for new output to be displayed
    data_dict = {key: widget.value for key, widget in input_widgets.items()}
    display_markdown()  # Display updated markdown
    for widget in input_widgets.values():  # Redisplay input widgets for further updates
        display(widget)
    display(update_button)  # Redisplay the update button
    display(cancel_button)  # Redisplay the cancel button

# Cancel function
def cancel_update(b):
    clear_output(wait=True)
    for key, widget in input_widgets.items():
        widget.value = data_dict[key]  # Reset to original values
    display_markdown()  # Redisplay original data_dict
    for widget in input_widgets.values():  # Redisplay input widgets
        display(widget)
    display(update_button)
    display(cancel_button)

update_button.on_click(update_data)
cancel_button.on_click(cancel_update)



    ### Experiment Data
    **Dataset:** fc  

    **Participants:** 1,2,3,4  

    **Runs:** 1,2,3,4  

    **Otro:** 111  
    

Text(value='fc', description='Dataset:')

Text(value='1,2,3,4', description='Participants:')

Text(value='1,2,3,4', description='Runs:')

Text(value='111', description='Otro:')

Button(description='Update', style=ButtonStyle())

Button(description='Cancel', style=ButtonStyle())


    ### Experiment Data
    **Dataset:** fc  

    **Participants:** 1,2,3,4  

    **Runs:** 1,2,3,4  

    **Otro:**   
    

AttributeError: 'Text' object has no attribute 'on_event'

In [40]:
forinput_widgets.values()

fc


AttributeError: 'Text' object has no attribute 'on_event'

'fc'

In [ ]:
data_dict

{'Dataset': 'fco',
 'Participants': '1,2,3,4,5,6',
 'Runs': '1,2,3,4',
 'Otro': ''}